In [1]:
import json
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from modules.faiss_retriever import BookRetriever

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

book_info_path = 'database/book_info.json'

try:
    with open(book_info_path, 'r', encoding='utf-8') as file:
        book_info = json.load(file)
except Exception as e:
    print(f"书籍数据加载失败：{e}")

TPM_limit = 1000000

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

book_retriever = BookRetriever("三体", embeddings, book_info_path)

模块已加载
成功加载书籍: 三体


In [ ]:
# 加载小说进Retriever
import tiktoken
novel_path = "novels/TOP/《三体》（实体版1-3全本）作者：刘慈欣.txt"
encoding = tiktoken.encoding_for_model("text-embedding-3-small")
loader = TextLoader(novel_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(
        separator='\n',
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
)
docs = text_splitter.split_documents(documents)
book_retriever.add_large_documents(docs, encoding, TPM_limit=TPM_limit)
book_retriever.save_local()

In [2]:
book_retriever.search("叶文洁是谁")

[(Document(metadata={'source': 'novels/TOP/《三体》（实体版1-3全本）作者：刘慈欣.txt'}, page_content='“不，那哪行！”白沐霖连连摆手说，“你们建设兵团的女战士，白天干的都是男同志的活儿，快回去休息吧，明天六点就要上山呢。哦，文洁，我后天就要回师部了，我会把你的情况向上级反映一下，也许能帮上忙呢。”\n\u3000\u3000“谢谢，不过我觉得这里很好，挺安静的。”文洁看着月光下大兴安岭朦胧的林海说。\n\u3000\u3000“你是不是在逃避什么？”\n\u3000\u3000“我走了。”叶文洁轻声说，转身离去。\n\u3000\u3000白沐霖看着她那纤细的身影在月光下消失，然后，他抬头遥望文洁刚才看过的林海，看到远方的雷达峰上，巨大的天线又缓缓立起，闪着金属的冷光。\n\u3000\u3000三个星期后的一天中午，叶文洁被从伐木场紧急召回连部。一走进办公室，她就发现气氛不对，连长和指导员都在，还有一个表情冷峻的陌生人，他面前的办公桌上放着一个黑色的公文包，旁边两件东西显然是从公文包中拿出来的，那是一个信封和一本书，信封是拆开的，书就是那本她看过的《寂静的春天》。\n\u3000\u3000这个年代的人对自己的政治处境都有一种特殊的敏感，而这种敏感在叶文洁身上更强烈一些，她顿时感到周围的世界像一个口袋般收紧，一切都向她挤压过来。\n\u3000\u3000“叶文洁，这是师政治部来调查的张主任，”指导员指指陌生人说， “希望你配合，要讲实话。”\n\u3000\u3000“这封信是你写的吗？”张主任问，同时从信封中抽出信来。叶文洁伸手去拿，但张主任没给她，仍把信拿在自己手中，一页一页翻给她看，终于翻到了她想看的最后一页，落款上没有姓名，只写着“革命群众”四个字。\n\u3000\u3000“不，不是我写的。”文洁惊恐地摇摇头。\n\u3000\u3000“可这是你的笔迹。”\n\u3000\u3000“是，可我是帮别人抄的。”\n\u3000\u3000“帮谁？”\n\u3000\u3000平时在连队遇到什么事，叶文洁很少为自己申辩，所有的亏都默默地吃了，所有的委屈都默默地承受，更不用说牵连别人了。但这次不同，她很清楚这意味着什么。\n\u3000\u3000“是帮那位上星期到连队来采访的《大生产报

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4o-mini')
retriever = book_retriever.vector_store.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_process_template = """请将以下内容总结为精炼版本，以结构化形式输出，重点保留主要人物的姓名、他们之间的关系、以及重要的地名或组织名。请回答以下问题：1. 谁是主要人物？2. 他们之间的关系是什么？3. 有哪些关键事件或地点？
'''{context}'''
"""
rag_process_prompt = ChatPromptTemplate.from_template(rag_process_template)

outline_template = """根据以上已知的角色的过往历史，与续写内容无直接联系，以用户给出要求进行续写，创作一个详细的续写大纲，以结构化形式输出。请确保大纲包括以下要素：1. 主要情节发展；2. 角色设定和发展；3. 背景设定；4. 主要冲突和解决方案。请考虑如何将历史内容与续写开头有效结合，并回答以下问题：主要人物面对的挑战是什么？故事的核心冲突是什么？
'''{context}'''
以下是用户提出的要求：
'''{question}'''
"""
outline_prompt = ChatPromptTemplate.from_template(outline_template)

expand_template = """请根据以下大纲创作一段故事
'''{context}'''
"""
expand_template = ChatPromptTemplate.from_template(expand_template)

rag_process_chain = (
    {"context": retriever | format_docs}
    | rag_process_prompt
    | llm
    | StrOutputParser()
)
outline_chain = (
    {"context": rag_process_chain, "question": RunnablePassthrough()}
    | outline_prompt
    | llm
    | StrOutputParser()
)
expand_chain = (
    {"context": outline_chain}
    | expand_template
    | llm
    | StrOutputParser()
)


In [66]:
# 绘制结构
from PIL import Image
from io import BytesIO

data = expand_chain.get_graph().draw_png()
image = Image.open(BytesIO(data))
image.show()

d:\anaconda3\envs\qqbot\Lib\site-packages\pygraphviz\agraph.py:1409: RuntimeWarning: Error: Unknown HTML element <context> on line 1 
in label of node 6ae3013e159d422889517b163aa36772
Error: not well-formed (invalid token) in line 1 
... Parallel<context,question> ...
in label of node 1298ce4ca02a4882b5005837519fa04e
Error: not well-formed (invalid token) in line 1 
... Parallel<context,question> ...
in label of node 9e2b30044bca492babed7d2302719c17

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


In [12]:
expand_chain.invoke("我希望续写故事是以 艾AA和云天明被困在黑域之中 为开头")

'在无尽的黑域中，艾AA和云天明漂浮在一片漆黑的宇宙中，四周没有任何光亮，只有沉重的寂静和无尽的恐惧。黑域的存在仿佛将他们的心灵也吞噬，让他们在这未知的空间里感到无助与迷茫。\n\n“我们必须找到出路。”艾AA的声音在黑暗中显得尤为坚定，她抬起头，凝视着无边的黑暗，似乎在努力寻找那一丝光亮。\n\n云天明微微颤抖，身体里的虚弱感如潮水般涌来。他努力回忆起与程心的点点滴滴，那是他生命中最美好的时光。然而，病痛和绝望渐渐侵蚀着他的勇气。他自言自语道：“我们可能永远也出不去。”\n\n“别这样想！”艾AA转过身来，目光坚定地锁定在云天明的眼中，“我们还活着，还有希望。你要相信，生命是值得的。”\n\n她的话如同一束光，穿透了云天明心中的黑暗。他闭上眼，努力将那些绝望的情绪压回心底，心中不断回响着程心温柔的声音：“活下去，无论多么艰难。”\n\n他们开始探索周围的环境，艾AA通过手中的科技设备，逐渐收集到一些数据。就在这时，前方突然闪现出一些微弱的光点，如星星般在黑域的深处闪烁。云天明的心跳加速，仿佛看到了希望的曙光。\n\n“我们去那些光点！”艾AA兴奋地说，拉着云天明的手，朝着光点的方向飞去。\n\n在接近光点的过程中，云天明的身体再一次被虚弱感击倒，他几乎无法继续前行。艾AA察觉到他的挣扎，立刻停下脚步，紧紧握住他的手：“不要放弃，我在你身边，我们一起走出这里！”\n\n此时的云天明，心中虽有犹豫与恐惧，但艾AA的坚强与勇气如同温暖的阳光，驱散了他内心的阴霾。他深吸一口气，努力站稳脚步，仿佛感受到了程心的微笑与鼓励。\n\n“我明白了，活着就是希望。”云天明低声说道，目光坚定，向着光点迈出坚定的一步。\n\n随着他们的接近，那些光点逐渐变得明亮，仿佛在为他们指引出路。就在这时，云天明脑海中浮现出程心的身影，她的眼中闪烁着对生活的热爱与不屈的精神，仿佛在告诉他，无论多么艰难，都要坚持下去。\n\n最终，云天明和艾AA抵达了光点的边缘，那里散发着温暖的光辉，仿佛是通往另一个世界的入口。云天明感受到了一种久违的力量，他的心中充满了对生命的渴望与希望。\n\n“我们可以做到！”艾AA微笑着，带着云天明一起迈入那片光明，黑域的阴霾在他们身后渐渐消散。\n\n在未来的旅途上，云天明与艾AA并肩而行，面对未知的挑战，他们的心灵在黑暗中交汇，彼此支持，寻找着生命中最耀眼的光芒。'